# Customised Kernel of the ACBC (only trained with the score tensor generated from Integrated model.ipynb)

In [9]:
from sklearn.metrics import classification_report



In [10]:
class MyModel:
    def __init__(self, w, lr):
        self.w = w
        self.size = w.shape[0]
        self.lr = lr
        
    def predict(self, x):
        length = x.shape[0]
        y_hat = np.zeros((length))
        for i in range(length):
            xi = x[i]
            w_masked = np.array([0 if np.isnan(xi[j]) else self.w[j] for j in range(self.size)])
            x_masked = np.array([0 if np.isnan(xij) else xij for xij in xi])
            w_sum = np.sum(w_masked)
            y_hat[i] = 0 if np.sum(x_masked * w_masked) / w_sum < 0.5 else 1
        return y_hat
    
    def GD_kernel(self, x,y):
        x_checking = x[np.isnan(x)]
        if x_checking.shape[0] == x.shape[0]:
            return 0, np.repeat(0, 36)
        
        W = self.w
        w_masked = np.array([0 if np.isnan(x[i]) else W[i] for i in range(self.size)])
        x_masked = np.array([0 if np.isnan(xi) else xi for xi in x])
        wx = w_masked * x_masked
        mask = np.array([0 if np.isnan(xi) else 1 for xi in x])
        sum_w = np.sum(w_masked)
        sum_wx = np.sum(wx)
        p = sum_wx / sum_w
        d_p = (x_masked*sum_w - sum_wx)/(sum_w**2) * mask
        loss = (y*(1-p) + (1-y)*p)
        gradients = -2*y*d_p + d_p
        gradients = gradients * mask

        label = 0.0 if p <0.5 else 1.0
        label = 1 if label == y else 0
        return label, gradients
        
    def train(self,x,y,epochs):
        length = x.shape[0]
        prev_w = self.w
        for i in range(epochs):
            avg_loss = 0
            gradients = np.zeros((length, self.size))
            for j in range(length):
                loss, g = self.GD_kernel(x[j], y[j])
                if loss == 0:
                    gradients[i] = g
                avg_loss += loss
                gradient = np.sum(gradients, axis = 0)
                self.w = self.w - gradient*self.lr
            print(self.w)
            print('acc in epoch',i,':',float(avg_loss)/float(length))
    
                

In [11]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
import pandas as pd

vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

def test_report(vehicle_type, train, test):
    print(train.shape)
    print('summary of test accuracy for vehicle type:', vehicle_type)
    model = MyModel(np.ones((36)),0.0002)
    model.train(train[:, 0:36], train[:,36],10)
    scores = model.predict(test[:,0:36])
    print(scores.shape[0])
    print(scores[scores==1].shape[0])
    #scores = np.mean(test[:,0:36],axis=1)
    print(scores.shape)
    #scores = np.array([1 if s >= 0.5 else 0 for s in scores])
    acc = accuracy_score(test[:,36], scores)
    
    print(classification_report(test[:,36], scores, digits=4))
    print('acc:', acc)
    fpr, tpr, thresholds = metrics.roc_curve(test[:,36], scores, pos_label=1)
    print('AUC:',metrics.auc(fpr, tpr))
    correct = int(acc * test.shape[0])
    print(correct,'/',test.shape[0])
    return correct

avg_acc = 0
for i in range(1):
    path = '../data/final/feature_tensors/'
    print(path)
    train_tensor = dict()
    test_tensor = dict()
    for vehicle_type in vehicle_types:
        train_tensor[vehicle_type] =pd.read_csv(path+vehicle_type+'_train.csv',sep=',',header=None).to_numpy()
        test_tensor[vehicle_type] = pd.read_csv(path+vehicle_type+'_test.csv', sep=',',header=None).to_numpy()



    correct = 0
    nums = 0
    for vehicle_type in vehicle_types:
        nums += test_tensor[vehicle_type].shape[0]
        correct += test_report(vehicle_type, train_tensor[vehicle_type], test_tensor[vehicle_type])
    avg_acc += (correct / nums)
    print('Test acc:', correct / nums)
print('average accuracy:', avg_acc)

../data/final/feature_tensors/
(13883, 37)
summary of test accuracy for vehicle type: ZVe44
[0.98184169 0.93394808 1.00284979 1.00076328 1.000039   1.00071977
 1.00022133 1.         1.         1.         1.         1.
 1.02592945 1.02498896 1.00055318 0.99992169 0.9956929  0.90854824
 1.00225583 1.00054577 1.00909531 1.01358105 0.99991108 1.
 1.00016383 1.         1.         1.         1.00365677 1.00831526
 1.         1.         1.04193225 1.03358318 1.00185059 1.00034376]
acc in epoch 0 : 0.8490960167110855
[0.95991923 0.86657597 1.0058694  1.00152845 1.00005426 1.00144009
 1.00044406 1.         1.         1.         1.         1.
 1.05088569 1.04807887 1.00114443 0.99984095 0.98756701 0.81656168
 1.00450809 1.00108748 1.01818945 1.02703891 0.99981897 1.
 1.00032633 1.         1.         1.         1.00695338 1.01660051
 1.         1.         1.08139497 1.06461344 1.00381617 1.00068912]
acc in epoch 1 : 0.8545703378232371
[0.9336635  0.80056431 1.00869708 1.00238806 1.00006831 1.0023

[0.01218855 0.60939603 0.9092269  1.09007682 1.0360448  1.02346811
 0.999803   1.         1.00100541 1.         1.         1.
 1.26701201 1.06207671 1.16448949 1.02404198 0.85245517 0.33512355
 0.59885848 1.0378004  1.21091893 1.19746908 1.03830712 1.01438707
 1.0018117  1.00005658 1.         1.         1.06578666 1.0681084
 1.00910543 1.00414161 1.00931256 1.1988609  1.10257344 1.0478557 ]
acc in epoch 8 : 0.8471733783261847
[-0.01352702  0.59928187  0.89162046  1.09353469  1.03841392  1.02527079
  0.99978138  1.          1.00119308  1.          1.          1.
  1.27487312  1.05238735  1.17118058  1.02636718  0.83368268  0.31776082
  0.5591795   1.0376943   1.2298964   1.21192331  1.0418784   1.01574972
  1.0020427   1.00005658  1.          1.          1.07130584  1.07425922
  1.01019246  1.00465663  0.99446208  1.19182416  1.11120221  1.05042905]
acc in epoch 9 : 0.8485760169129755
12299
6355
(12299,)
              precision    recall  f1-score   support

         0.0     0.6691    0

[0.95279942 0.97829971 1.0091531  1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.00342417 1.02958705 1.         1.         0.98660496 1.01443336
 1.0118049  1.         1.         1.00549022 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.01140509 0.99569983 1.00001254 1.        ]
acc in epoch 2 : 0.9892292963140259
[0.9366841  0.97087808 1.01217932 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.00457093 1.03937969 1.         1.         0.98189085 1.0192012
 1.01571897 1.         1.         1.00732313 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.01521518 0.9942305  1.00001667 1.        ]
acc in epoch 3 : 0.9892292963140259
[0.92145681 0.96298503 1.01519273 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.00616628 1.04913886 1.         1.         0.97713069 1.02401094
 1

[1.00839507 1.         1.01265012 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 0.99019291 1.         1.         1.         0.99019291 1.
 0.99842722 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.        ]
acc in epoch 2 : 0.9971014492753624
[1.01117459 1.         1.01684803 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 0.98690489 1.         1.         1.         0.98690489 1.
 0.99788404 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.        ]
acc in epoch 3 : 0.9971014492753624
[1.01394474 1.         1.0210366  1.         1.         1.
 1.         1.         1.         1.         1.         1.
 0.98360732 1.         1.         1.         0.98360732 1.
 0.99733139 1.         1.